In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb

The file_gams_py_gdb0.gdx is still active and was not deleted.


*A few local functions:*

In [2]:
def appmap_s(s,mi,level=None,add_new_name=False): # auxiliary function used for matching indices
    return DataBase_wheels.mi.v3_series(s,mi,level=level,add_new_name=add_new_name)

In [3]:
def unionlist(mi,l=None):
    if l is None:
        return unionlist(mi[0], l=mi[1:])
    elif len(l)>1:
        return unionlist(mi.union(l[0]),l=l[1:])
    else:
        return mi.union(l[0])

# Partial equilibrium model with technologies, v 0.1 [(link to paper)](https://www.overleaf.com/read/stkxjdbsyzjq)

In [4]:
name = 'V01'

We use the following GAMS variable conventions:
* ```qD[s,n]```: Quantity of demand for good $n$ in sector $s$. ($\color{blue}{\text{read from data}}$)
* ```qS[s,n]```: Quantity of supply of good $n$ in sector $s$. ($\color{blue}{\text{read from data}}$)
* ```pD[s,n]```: Price paid for good $n$ by sector $s$ (includes taxes etc.). ($\color{blue}{\text{partially read from data}}$, $\color{red}{\text{CHECK}}$.)
* ```pS[s,n]```: Cost price of good $n$ in sector $s$ (no taxes, mark-ups etc. included). $\color{red}{\text{CHECK}}$.
* ```mu[s,n,nn]```: Share parameters. ($\color{blue}{\text{read from data}}$)
* ```sigma[s,n]```: price elasticity. ($\color{blue}{\text{read from data}}$)

Including technologies we extend this namespace convention:
* ```qD[s,n]```: When $n$ is a technology it refers to the share of its potential that is realized, i.e. $qD[s,n]\in[0,1]$. ($\color{blue}{\text{read from data}}$)
* ```mu[s,n,nn]```: When $n$ is a technology it refers to the shadow cost parameter used to calibrate the current use of a technology. Without calibration $\mu[s,n,nn] = 0$. ($\color{blue}{\text{read from data}}$)
* ```sigma[s,n]```: When $n$ is a technology it is a measure of cost heterogeneity of the technology. ($\color{blue}{\text{read from data}}$)
* ```theta[s,n]```: Only defined for technology goods $n$. Measures potential of technology, $theta[s,n]\in[0,1]$. ($\color{blue}{\text{read from data}}$)
* ```lambda[s,n]```: Only defined for energy services $n$. Measures marginal shadow cost of producing $n$ using the available technologies. $\color{red}{\text{CHECK}}$.

The implication is that the set $n$ is quite a large one that also encompasses intermediate goods. The relevant GAMS sets/subsets/mappings used throughout the code will be:
* ```{name}_map[s,n,nn]```: Maps branches (n) to knots (nn) in the entire nesting production tree. $\color{red}{\text{CHECK}}$.
* ```{name}_NT_out[s,n]```: Subset of goods (n) that are outputs from sector (NT refers to Non-Technology part). $\color{red}{\text{CHECK}}$
* ```{name}_NT_bra_out[s,n]```: Subset of goods (n) that are branches in the NT part of the nesting tree, and the corresponding knot is an output from the sector (s). $\color{red}{\text{CHECK}}$
* ```{name}_NT_bra_nout[s,n]```: Subset of goods (n) that are branches in the NT part of the nesting tree, and the corresponding knot is not an output from the sector (s). $\color{red}{\text{CHECK}}$
* ```{name}_NT_int[s,n]```: subset of goods (n) that are intermediate goods and included in the NT part of nesting tree. $\color{red}{\text{CHECK}}$
* ```{name}_NT_inp[s,n]```: subset of goods (n) that are inputs in the NT part of the nesting tree. This includes specific elements like ```M_X1```, but not sector aggregates ```X1```. $\color{red}{\text{CHECK}}$
* ```{name}_NT_map[s,n,nn]```: Part of ```{name}_map[s,n,nn]``` in the NT part of the nesting tree. $\color{red}{\text{CHECK}}$.
* ```{name}_ES[s,n]```: Subset of goods (n) that are energy services. $\color{red}{\text{CHECK}}$
* ```{name}_T[s,n]```: Subset of goods (n) that are technology goods. $\color{red}{\text{CHECK}}$
* ```{name}_inp2T[s,n,nn]```: Mapping from inputs (n) to technology goods (nn). $\color{red}{\text{CHECK}}$.
* ```{name}_dur[s,n]```: Subset of goods (n) that are durables in the sector. ($\color{blue}{\text{read from data}}$)
* ```{name}_T2ES[s,n,nn]```: Mapping from energy services (n) to technology goods (nn). $\color{red}{\text{CHECK}}$.
* ```{name}_T2ESNorm```: Elements in T2ES for which the shadow-cost is normalized at zero. $\color{red}{\text{CHECK}}$.
* ```{name}_inp[s,n]```: Subset of goods (n) that are inputs. Includes aggregates ```X1```, but not specific elements like ```M_X1```. $\color{red}{\text{CHECK}}$.
* ```{name}_x2inp[s,n,nn]```: Mapping from specific values (n) to aggregate valutes (nn) - e.g. ```(M_X1, X1)``` ($\color{blue}{\text{read from data}}$)
* ```{name}_NT_x[s,n]```: Subset of specific values (n) from ```x2inp```. $\color{red}{\text{CHECK}}$.

Naming conventions: 
* We use ```xx``` (or similarly ```xxx``` if needed) to denote aliases of a set ```x```.
* Mappings are named ```X2Y``` when mapping from ```X``` to ```Y```.

## 1. Data

Load tech data / other data separately (helps automatically define some subsets):*

In [5]:
dataf = directory['data']+'\\Nesting_V01.xlsx'
data = excel2py.xl2PM.pm_from_workbook(dataf, {'NonTech': 'vars', 'Maps': 'maps'})
tech = excel2py.xl2PM.pm_from_workbook(dataf,{'Tech': 'vars'})

### 1.1. Non-Technology part

In [6]:
nt = name+'_NT'

*Maps, inputs, outputs, intermediate goods from nesting structure:*

In [7]:
def inp_from_map(map_):
    return map_.droplevel('nn')[map_.droplevel('nn').get_level_values('n').isin(set(map_.get_level_values('n'))-set(map_.get_level_values('nn')))]
def out_from_map(map_):
    return map_.droplevel('n')[map_.droplevel('n').get_level_values('nn').isin(set(map_.get_level_values('nn'))-set(map_.get_level_values('n')))].unique().set_names(['s','n'])
def int_from_map(map_,inp):
    return map_.droplevel('nn')[~map_.droplevel('nn').isin(inp)].unique()

In [8]:
data[f"{nt}_map"] = data['mu'].index
data[f"{nt}_inp"] = inp_from_map(data[f"{nt}_map"].vals)
data[f"{nt}_out"] = out_from_map(data[f"{nt}_map"].vals)
data[f"{nt}_int"] = int_from_map(data[f"{nt}_map"].vals, data[f"{nt}_inp"].vals)

*Split branches in nesting tree into subset that are connected to an output knot and intermediate knot:*

In [9]:
data[f"{nt}_bra_out"] = data[f"{nt}_map"].rctree_pd(gpy_symbol(data[f"{nt}_out"].vals.set_names(['s','nn']))).droplevel('nn').unique() # branches in map where corresponding knot is an output.
data[f"{nt}_bra_nout"] = gpy_symbol(data[f"{nt}_int"].vals.union(data[f"{nt}_inp"].vals)).rctree_pd({'not': data[f"{nt}_bra_out"]}).unique() # intermediates/inputs not in bra_out.

### 1.2. Technology part

In [10]:
t = name

*Energy services are outputs from the technology tree. Technologies are intermediates.*

In [11]:
tech[f"{t}_ES"] = out_from_map(tech['mu'].index)
tech[f"{t}_T"] = int_from_map(tech['mu'].index, inp_from_map(tech['mu'].index))

*inp2T are the part of the mapping where knots are technologies. T2ES is the rest of the tech tree:*

In [12]:
tech[f"{t}_inp2T"] = tech['mu'].rctree_pd({'not': tech[f"{t}_T"]}).index
tech[f"{t}_T2ES"] = tech['mu'].rctree_pd(tech[f"{t}_T"]).index

*T2ESNorm picks out a single element in the mapping for each E:*

In [13]:
tech[f"{t}_T2ESNorm"] = pd.MultiIndex.from_frame(tech[f"{t}_T2ES"].vals.to_frame(index=False).drop_duplicates(subset='nn',keep='first'))

### 1.3. Aggregates

*Inputs are defined by x2inp:*

In [14]:
data[f"{name}_inp"] = data['x2inp'].vals.droplevel('n').unique().set_names(['s','n'])
data[f"{nt}_x"] = data['x2inp'].vals.droplevel('nn').unique()

*For the rest of the elements in the non-technology tree, expand x2inp to include (x,x) mappings:*

In [15]:
temp = gpy_symbol(data[f"{nt}_inp"].vals.union(data[f"{nt}_int"].vals)).rctree_pd({'not': gpy_symbol(data['x2inp'].vals.droplevel('nn'))}) # all intermediates or inputs in NT that are not in x2inp

*For the rest of the elements in the non-technology tree, expand x2inp to include (x,x) mappings:*

In [16]:
data[f"{name}_x2inp"] = data['x2inp'].vals.union(pd.MultiIndex.from_frame(temp.to_frame(index=False).assign(nn=temp.get_level_values('n')))).unique()

*clean up namespace:*

In [17]:
data[f"{name}_dur"] = data['dur'] # add specific name to durables
[data.__delitem__(s) for s in ('x2inp','dur')]; # delete local copies

*Add/merge technology data*

In [18]:
DataBase.GPM_database.merge_dbs(data,tech,'second')

*Define entire nesting tree map from share parameters:*

In [19]:
data[f"{name}_map"] = data['mu'].index

### 1.4. Initial values

*Compute $\lambda$ from data given logit formula in paper:*

In [20]:
Ts = gpy_symbol(data[f"{name}_T2ESNorm"].vals.droplevel('nn')) # relevant normalized T's
T2E = data[f"{name}_T2ESNorm"].index.droplevel('s') # Map from T2E:
data['lambda'] = (appmap_s(data['pD'].rctree_pd(Ts),T2E)+data['sigma'].rctree_pd(data[f"{name}_ES"])*appmap_s((data['qD'].rctree_pd(Ts)/(1-data['qD'].rctree_pd(Ts))).astype(float).apply(np.log), T2E)).rename('lambda')

*Compute shadow values from logit formula:*

In [21]:
Ts = gpy_symbol(gpy_symbol(data[f"{name}_T2ES"].vals.droplevel('nn')).rctree_pd({'not': Ts}))
sigma_Tij = (data['sigma'].rctree_pd(data[f"{name}_ES"])*pd.Series(1, index = data[f"{name}_T2ES"].rctree_pd(Ts).set_names(['s','nn','n']))).droplevel('n').rename_axis(index={'nn':'n'})
lambda_Tij = (data['lambda'].vals * pd.Series(1, index = data[f"{name}_T2ES"].rctree_pd(Ts).set_names(['s','nn','n']))).droplevel('n').rename_axis(index={'nn':'n'})
logshare = (data['qD'].rctree_pd(Ts)/(1-data['qD'].rctree_pd(Ts))).astype(float).apply(np.log)
shadowcosts = DataBase_wheels.mi.add_mi_series(data['pD'].rctree_pd(Ts)+sigma_Tij*logshare-lambda_Tij, data["V01_T2ES"].rctree_pd(Ts),level='n').rename('mu')
data['mu'].vals = shadowcosts.combine_first(data['mu'].vals)

*Compute share parameters on capital for technologies (this only works when there is one durable):*

In [22]:
K = data[f"{name}_dur"] # capital subset
nK = gpy_symbol(data[f"{name}_inp"].rctree_pd({'not': data[f"{name}_dur"]})) # not capital, inputs
inp2T_notK = gpy_symbol(data[f"{name}_inp2T"].rctree_pd(nK)) # mapping from inputs to technologies, not capital
inp2T_K = gpy_symbol(data[f"{name}_inp2T"].rctree_pd(K)) # mapping from inputs to technologies, only capital
costs_notK = (data['mu'].rctree_pd(inp2T_notK) * (data['pD'].rctree_pd(nK))).groupby(['s','nn']).sum().rename_axis(index={'nn':'n'}) # sum of costs related to non-capital inputs for each technology.
mu_K = (pd.Series(1, index = data['mu'].rctree_pd(inp2T_K).index) * (((data['pD'].rctree_pd(data[f"{name}_T"])-costs_notK)/data['pD'].rctree_pd(K).droplevel('n')).rename_axis(index={'n':'nn'}))).rename('mu')
if min(mu_K)<0:
    warnings.warn("Technology and price data implies negative coefficients on capital.")
data['mu'].vals = mu_K.combine_first(data['mu'].vals)

*If prices are not read in from excel sheets, just set them to 1:*

In [23]:
data['pS'] = pd.Series(1, index = data['qS'].index, name = 'pS')
data['pD'] = data['pD'].vals.combine_first(pd.Series(1, index = unionlist([data[f"{nt}_int"].vals, data[f"{name}_inp"].vals, data[f"{name}_ES"].vals, data[f"{name}_T"].vals]), name = 'pD')) # if there is not entry yet, set 1

*Compute the implied level of quantites of inputs used outside the technology trees:*
1. *Compute the shares for each technology/input type:* $\mu_l^{i,j}\tau_i^j\theta_i^j$
2. *Map this to energy services, multiply by $E_j$ and sum.*
3. *Define residual demand as initial levels used outside technology sector.*

In [24]:
inp_T = data['mu'].rctree_pd(data[f"{name}_inp2T"]).rename_axis(index=['s','nn','n'])* (data['qD'].rctree_pd(data[f"{name}_T"]) * data['theta'].vals) # step 1.
qD_Tech = (appmap_s(inp_T,data[f"{name}_T2ES"].vals.droplevel('s'))*data['qD'].rctree_pd(data[f"{name}_ES"])).groupby(['s','nn']).sum()
res_demand = (data['qD'].rctree_pd(data[f"{name}_inp"])-qD_Tech.rename_axis(['s','n'])).rename_axis(['s','nn'])
if min(res_demand)<=0:
    warnings.warn("Inputs used in technology nests and total sector inputs implies negative demand outside technology nests.")
data['qD'].vals = appmap_s(res_demand, data[f"{name}_x2inp"].vals.droplevel('s'),add_new_name=True).rename('qD').combine_first(data['qD'].vals)

*Set up database and export*

In [25]:
db = DataBase.GPM_database(alias = pd.MultiIndex.from_tuples([('n','nn'),('n','nnn')]), **{'name': f"{name}_DB", 'data_folder': directory['data']})
db.merge_dbs(db.series, data, 'second') # add data 
db.update_all_sets(types=['variable','parameter','mapping']) # read definition of 's','n' from what appears in data
db.export()